# Generating dummy data

## Import dependencies

In [2]:
# Import dependencies
from faker import Faker
import numpy as np
import csv
import datetime
from collections import OrderedDict
import pandas as pd
import datetime as dt
import pandas as pd

In [3]:
# Instantiate random models
fake = Faker()
Faker.seed(2203)
rng = np.random.default_rng(seed = 2203)

In [4]:
# Set amount of data to generate
n_inventory = 5000

n_vendors = 100
n_employees = 50

n_games = 1250
n_movies = 3750

# The combined number of movies and games cannot exceed the total inventory
assert n_games + n_movies == n_inventory, 'Games and movies cannot exceed inventory!'

n_castmembers = 1000

# Kiosks does not have a count, but should be dependent on the number of hosts
# Fewer hosts --> higher mu
n_hosts = 100
mu_kiosks = 5
sd_kiosks = 1

n_customers = 100000
percent_borrowing = 75 # Int that represents what percentage of customers currently have a rental

n_paymentmethods = 150000
assert n_paymentmethods >= n_customers, 'Every customer should have at least one payment method on file!'

## Make tables

### Inventory table 
**Documentation**: `title` is a randomly generated text string that has a randomly determined number of characters. `release_year` is randomly generated year between 1950 and 2023.

- inventory_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- title varchar(50) NOT NULL,
- release_year year NOT NULL

In [5]:
INVENTORY = [['title', 'release_year', 'movie']]

# Generate some fake inventory
for i in range(n_inventory) :
    INVENTORY.append([fake.text(max_nb_chars = rng.integers(5,30)), 
                      fake.date_between_dates(date_start = datetime.date(1950, 1, 1),
                                              date_end = datetime.date(2023, 12, 31)).year,
                      int(fake.boolean(75))])


# Write to CSV
with open('Final Project/Phase 2/Fake Data/inventory.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(INVENTORY)

### Vendors table
**Documentation**: `vendor_name` is a randomly generated company name. `vendor_st_address`, `vendor_zip_code`, `vendor_phone`, and `vendor_email` are randomly generated street addresses, zip codes, phone numbers, and emails (respectively). 
- vendor_name
- vendor_st_address
- vendor_zip_code
- vendor_phone
- vendor_email

In [6]:
VENDORS = [['vendor_name', 'vendor_st_address', 'vendor_zip_code', 'vendor_phone', 'vendor_email']]

# Generate some fake vendors
for i in range(n_vendors) :
    person = fake.simple_profile()

    VENDORS.append([fake.company(), fake.street_address(), fake.postcode(), fake.phone_number(), person['mail']])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/vendors.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerows(VENDORS)

### Employees table
**Documentation**: `employee_fname` is a randomly generated nonbinary first name. The gender of the first name might not match the `gender` variable. `employee_lname` is a randomly generated last name. `employee_gender` is randomly assigned as M or F (for male/female). `employee_st_address`, `employee__zip_code`, `employee__phone`, and `employee__email` are randomly generated street addresses, zip codes, phone numbers, and emails (respectively). `employee_auth_code` is a random 5-digit integer. `employee_auth_code` is not checked for uniqueness since the number of employees is small compared the the number of possible authorization codes. `employee_title` is a randomly generated job.
- employee_fname
- employee_lname
- employee_st_address
- employee_zip_code
- employee_mobile
- employee_email
- employee_auth_code
- employee_title

In [7]:
# Define list and header row
EMPLOYEE = [['employee_fname', 'employee_lname', 'employee_gender', 'employee_st_address', 'employee_zip_code', 
             'employee_mobile_phone', 'employee_email', 'employee_auth_code', 'employee_title']]

# Generate some fake employees
for i in range(n_employees) :
    person = fake.simple_profile()

    EMPLOYEE.append([fake.first_name_nonbinary(), fake.last_name(), person['sex'], fake.street_address(), 
                     fake.postcode(), fake.phone_number(), person['mail'], rng.integers(10000,100000), 
                     fake.job()])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/employee.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerows(EMPLOYEE)

### Games table
**Documentation**: `game_publisher` is a randomly generated company. `game_platform` is a randomly selected variable from a list of popular gaming consoles. `game_esrb_rating` is a randomly selected variable from the five possible ESRB ratings given to video games. 
- inventory_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- game_publisher varchar(50) NOT NULL,
- game_platform varchar(50) NOT NULL,
- game_esrb_rating varchar(5) NOT NULL,
- FOREIGN KEY (inventory_id) REFERENCES Inventory(inventory_id)


In [8]:
GAMES = [['game_publisher', 'game_platform', 'game_esrb_rating']]

platforms = ['PS4', 'PS5', 'Xbox Series X/S', 'Xbox One', 'Nintendo Switch']
esrb_ratings = ['E', 'E10+', 'T', 'M', 'AO']

for i in range(n_games) :
    GAMES.append([fake.company(), fake.random_choices(elements = platforms, length = 1)[0], 
                     fake.random_choices(elements = esrb_ratings, length = 1)[0]])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/games.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerows(GAMES)

### Type table
**Documentation**: `type_desc` is a randomly selected variable from a list of possible types/genres of video games.
- type_id int NOT NULL PRIMARY KEY,
- type_desc varchar(50) NOT NULL 

In [9]:
TYPES = [['type_desc'], ['Shooters'], ['Adventure'], ['Strategy'], ['PvP'], ['RPG'], 
          ['Simulation'], ['Puzzle'], ['Horror'], ['War']]

# Write to CSV
with open('Final Project/Phase 2/Fake Data/type.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(TYPES)

### Game type table [BRIDGE]
**Documentation**: Games are identified by a value of `False` in the Movie column of the INVENTORY table. For each item in INVENTORY that is a game, 1-3 types are randomly assigned to the inventory key of that game.  
- inventory_id int NOT NULL,
- type_id int NOT NULL,
- CONSTRAINT PK_Game_Type PRIMARY KEY (inventory_id,type_id),
- CONSTRAINT FK_inventory_id FOREIGN KEY (inventory_id) REFERENCES Inventory(inventory_id),
- CONSTRAINT FK_type_id FOREIGN KEY (type_id) REFERENCES Type(type_id)


In [10]:
GAME_TYPE = [['inventory_id', 'type_id']]

type_ids = len(TYPES)

for i in range(len(INVENTORY)) :
    if INVENTORY[i][2] == False :
        type_count = rng.integers(1,4)
        types = []

        for j in range(type_count): 
            types.append(rng.integers(1,type_ids))
        
        for k in list(np.unique(types)) :
            GAME_TYPE.append([i, k])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/game_type.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(GAME_TYPE)


### Movies table
**Documentation**: `movie_runtime` is a random normally distributed integer ($X \sim N(\mu = 96, \sigma = 30$). `movie_mpaa_rating` is a randomly selected MPAA rating for a movie that follows a specified probability distribution that disadvantaged PG and NC-17 movies, as they are not as common. `movie_studio` is a randomly selected movie studio from a list of seven popular studios.`movie_plot` is a randomly generated 3-sentence paragraph. `bluray_available` is a randomly generated boolean variable.
- inventory_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- movie_runtime int NOT NULL,
- movie_mpaa_rating varchar(5) NOT NULL,
- movie_studio varchar(50) NOT NULL,
- movie_plot varchar(200) NOT NULL,
- bluray_available Boolean NOT NULL,
- FOREIGN KEY (inventory_id) REFERENCES Inventory(inventory_id)

In [11]:
MOVIES = [['movie_rutime', 'movie_mpaa_rating', 'movie_studio', 'movie_plot', 'bluray_available']]

movie_studios = ['Warner Bros.', 'Paramount Pictures', '20th Century Fox', 'Universal Pictures', 
                 'Columbia Pictures', 'A24', 'Blumhouse']
mpaa_ratings = [('G', .15), ('PG', .20), ('PG-13', .20), ('R', .20), ('NC-17', .05)]

for i in range(n_movies) :
    MOVIES.append([int(rng.normal(96,30)), 
                   fake.random_choices(elements=OrderedDict(mpaa_ratings), length = 1)[0], 
                   fake.random_choices(elements = movie_studios, length = 1)[0], fake.paragraph(2), 
                   int(fake.boolean(50))])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/movies.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(MOVIES)

### Genre table
**Documentation**: `type_desc` is a genre name/description. Genres are take from [Netflix's 'headline' categories](https://www.whats-on-netflix.com/news/the-netflix-id-bible-every-category-on-netflix/).
- genre_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- genre_desc varchar(50) NOT NULL

In [12]:
GENRE = [['genre_desc'], ['Action & Adventure'], ['Anime'], ['Children & Family'], ['Classic'], ['Comedies'], 
          ['Documentaries'], ['Dramas'], ['Horror'], ['Music'], ['Romantic'], ['Sci-fi & Fantasy'], 
          ['Sports'], ['Thrillers'], ['TV Shows']]

# Write to CSV
with open('Final Project/Phase 2/Fake Data/genre.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(GENRE)

### Movie Genre table [BRIDGE]
**Documentation**: Movies are identified by a value of `True` in the Movie column of the INVENTORY table. For each item in INVENTORY that is a movie, 1-3 genre are randomly assigned to the inventory key of that movie.    
- inventory_id int NOT NULL,
- genre_id int NOT NULL,
- CONSTRAINT PK_Movie_Genre PRIMARY KEY (inventory_id,genre_id),
- CONSTRAINT FK_inventory_id FOREIGN KEY (inventory_id) REFERENCES Inventory(inventory_id),
- CONSTRAINT FK_genre_id FOREIGN KEY (genre_id) REFERENCES Genre_TABLE(genre_id)

In [13]:
MOVIE_GENRE = [['inventory_id', 'genre_id']]

genre_ids = len(GENRE)

for i in range(len(MOVIES) - 1) :
    if INVENTORY[i][2] == True :
        genre_count = rng.integers(1,4)
        while genre_count > 0 :
            MOVIE_GENRE.append([i, rng.integers(1, genre_ids)])
            genre_count = genre_count - 1

# Write to CSV
with open('Final Project/Phase 2/Fake Data/movie_genre.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(MOVIE_GENRE)

### Cast members table
**Documentation**: `cast_fname` and `cast_lname` are randomly generated names.
- cast_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- cast_fname varchar(50) NOT NULL,
- cast_lname varchar(50) NOT NULL

In [14]:
CAST_MEMBERS = [['cast_fname', 'cast_lname']]

cast_types = [('Actor', .45), ('Writer', .45), ('Director', .10)]

# Generate some fake cast members
for i in range(n_castmembers) :
    person = fake.simple_profile()

    CAST_MEMBERS.append([fake.first_name(), fake.last_name(), 
                         fake.random_choices(elements=OrderedDict(cast_types), length = 1)[0]])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/cast_members.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(CAST_MEMBERS)

### Cast table [BRIDGE]
**Documentation**: Movies are identified by a value of `True` in the Movie column of the INVENTORY table. For each item in INVENTORY that is a movie, 1-6 cast members are randomly assigned to the inventory key of that movie.  
- cast_id int NOT NULL,
- inventory_id int NOT NULL
- CONSTRAINT pk_cast PRIMARY KEY (cast_id, inventory_id),
- CONSTRAINT fk_cast_id FOREIGN KEY (cast_id) REFERENCES CAST_MEMBERS(cast_id),
- CONSTRAINT fk_inventory_id FOREIGN KEY (inventory_id) REFERENCES INVENTORY(inventory_id)

In [15]:
CAST = [['cast_id', 'inventory_id']]

cast_ids = len(CAST_MEMBERS)

for i in range(len(INVENTORY)) :
    if INVENTORY[i][2] == True :

        cast_count = rng.integers(1,7)
        headliners = []

        for j in range(cast_count): 
            headliners.append(rng.integers(1,cast_ids))
        
        for k in list(np.unique(headliners)) :
            CAST.append([k, i])
        
# Write to CSV
with open('Final Project/Phase 2/Fake Data/cast.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(CAST)

### Host table
**Documentation**: `host_name` is a randomly generated company name. `host_st_address`, `host__zip_code`, `host__phone`, and `host__email` are randomly generated street addresses, zip codes, phone numbers, and emails (respectively). 
- host_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- host_name varchar(50) NOT NULL,
- host_st_address varchar(100) NOT NULL,
- host_email varchar(100) NOT NULL,
- host_phone char(10) NOT NULL,
- host_zip_code char(5) NOT NULL

In [16]:
HOST = [['host_name', 'host_st_address', 'host_email', 'host_phone', 'host_zip_code']]

# Generate some fake hosts
for i in range(n_hosts) :
    person = fake.simple_profile()

    HOST.append([fake.company(), fake.street_address(), person['mail'], fake.phone_number(), fake.postcode()])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/host.csv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(HOST)

### Kiosk table [BRIDGE]
**Documentation**: `kiosk_st_address` and `kiosk_zip_code`
- kiosk_id int NOT NULL PRIMARY KEY,
- kiosk_st_address varchar(100) NOT NULL,
- kiosk_zip_code char(5) NOT NULL,
- host_id int NOT NULL,
- CONSTRAINT FK_host_id FOREIGN KEY (host_id) REFERENCES Host(host_id)

In [17]:
KIOSK = [['kiosk_st_address', 'kiosk_zip_code', 'host_id']]

for i in range(1,len(HOST)) :
    kiosk_count = rng.normal(loc = mu_kiosks, scale = sd_kiosks)
    kiosk_count = kiosk_count if kiosk_count > 0 else 1

    while kiosk_count > 0 :
        KIOSK.append([fake.street_address(), fake.postcode(), i])
        kiosk_count = kiosk_count - 1

# Write to a CSV
with open('Final Project/Phase 2/Fake Data/kiosk.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(KIOSK)

In [18]:
print(len(KIOSK))

542


### Rental Inventory table [BRIDGE]
**Documentation**: `quantity` is a random integer between 1 and 4 that describes the number of DVDs available at the rental kiosk. `rental_rate` is the daily rate for renting the movie. Each  peice of rental inventory has an 85% chance of being stocked in a given rental kiosk, and for each piece of invebtory in a kiosk, the quantity is a randomly generated number between 1 and 4. 
- quantity int NOT NULL,
- rental_rate float NOT NULL,
- inventory_id int NOT NULL,
- kiosk_id int NOT NULL
- CONSTRAINT PK_Rental_Inventory PRIMARY KEY (inventory_id,kiosk_id),
- CONSTRAINT FK_inventory_id FOREIGN KEY (inventory_id) REFERENCES Inventory(inventory_id),
- CONSTRAINT FK_kiosk_id FOREIGN KEY (kiosk_id) REFERENCES Kiosk(kiosk_id)

In [19]:
RENTAL_INVENTORY = [['quantity', 'rental_rate', 'inventory_id', 'kiosk_id']]

for i in range(1,len(INVENTORY)) :
    for j in range(1,len(KIOSK)) :
        if fake.boolean(85) :
            quantity = rng.integers(1,5)
            RENTAL_INVENTORY.append([quantity, 1.00, i, j])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/rental_inventory.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(RENTAL_INVENTORY)



In [26]:
test_list = []
for i in range(len(RENTAL_INVENTORY)) :
    test_list.append(RENTAL_INVENTORY[3])

print(np.unique(test_list))

[1. 3. 4.]


### Customer table
**Documentation**: `customer_fname` is a randomly generated nonbinary first name. The gender of the first name might not match the `gender` variable. `customer_lname` is a randomly generated last name. `customer_gender` is randomly assigned as M or F (for male/female). `customer_st_address`, `customer_zip_code`, `customer_mobile_phone`, and `customer__email` are randomly generated street addresses, zip codes, phone numbers, and emails (respectively). `membership_status_id` is an random integer value between 0 and 4 that represents a specific tier of membership. The distribution of `membership_status_id` is skewed right to represent that fewer people are part of the higher tiers. `date_created` is a randomly generated datetime object between May 1st, 1991 and today. `customer_birthday` is a random datetime object. `customer_age_range` is a calculated field that categorizes customers by their age. 
- customer_fname, 
- customer_lname, 
- customer_st_address, 
- zip_code, 
- customer_mobile_phone
- customer_email, 
- membership_status_id,
- date_created
- customer_birthday
- customer_age_range

In [21]:
# Define list and header row
CUSTOMER = [['customer_fname', 'customer_lname', 'customer_gender', 'customer_st_address', 'zip_code', 
             'customer_mobile_phone', 'customer_email', 'membership_status_id', 'customer_date_created',
             'customer_birthday', 'customer_age_range']]

# Generate some fake customers
for i in range(n_customers) :
    person = fake.simple_profile()
    age = (pd.to_datetime('today').year - person['birthdate'].year)

    if age < 18 :
        age_range = 'Under 18'
    elif age >= 18 and age < 25 :
        age_range = '18-24'
    elif age >= 25 and age < 35 :
        age_range = '25-34'
    elif age >= 35 and age < 45 :
        age_range = '35-44'
    elif age >= 45 and age < 55 :
        age_range = '45-54'
    elif age >= 55 and age < 65 :
        age_range = '55-64'
    else : age_range = '65+'

    membership_statues = [(1, .10), (2, .35), (3, .35), (4, .15), (5, .05)]

    CUSTOMER.append([fake.first_name_nonbinary(), fake.last_name(), person['sex'], fake.street_address(), 
                     fake.postcode(), fake.phone_number(), person['mail'], 
                     fake.random_choices(elements=OrderedDict(membership_statues), length = 1)[0], 
                     fake.date_between(), person['birthdate'], age_range])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/customer.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(CUSTOMER)

### Payment Method table
**Documentation**: `card_provider` is a randomly generated credit card company. `card_number`, `exp_date` and `security_code` are randomly generated credit-card information.
- payment_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- FOREIGN KEY (customer_id) REFERENCES CUSTOMER(customer_id),
- card provider varchar(6) NOT NULL,
- card_number char(16) NOT NULL,
- exp_date datetime NOT NULL,
- security_code char(3) NOT NULL

In [23]:
PAYMENT_METHOD = [['customer_id', 'card_provider', 'card_number', 'exp_date', 'security_code']]

# Generate some fake credit cards
for i in range(1, len(CUSTOMER)) :
    PAYMENT_METHOD.append([i, fake.credit_card_provider(), fake.credit_card_number(), fake.credit_card_expire(),
                           fake.credit_card_security_code()])

# Write to CSV
with open('Final Project/Phase 2/Fake Data/payment_method.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(PAYMENT_METHOD)

### Rent table [BRIDGE]
**Documentation**: At any given time, about `percent_borrwing`% of our customers have at least one rental out, though they may have up to three rentals at a time. `rental_period` is a randomly selected integer from 1-7 that represents how many days the customer is charged to rent the movie. `rental_start_date` represents the date the customer first rented the movie. The difference between today (data generation date) and the start of the rental period is normally distributed ($X \sim N(\mu = 2, \sigma = .5)$). `price_charged` indicates how much money the customer was charged to rent the item initially (does not include late fees). `overdue` is a Boolean variable that indicates whether the customer has exceeded the rental period, and `overdue_duration` quantifies the time overdue. 

- inventory_id int NOT NULL,
- customer_id int NOT NULL,
- rental_period int NOT NULL,
- rental_start_date datetime NOT NULL,
- rental_end_time datetime NOT NULL,
- price_charged float NOT NULL,
- overdue Boolean NOT NULL,
- overdue_duration int NOT NULL,
- late_fee_total float NOT NULL,
- CONSTRAINT pk_rent PRIMARY KEY (inventory_id, customer_id),
- CONSTRAINT fk_inventory_id FOREIGN KEY (inventory_id) REFERENCES INVENTORY(inventory_id),
- CONSTRAINT fk_customer_id FOREIGN KEY (customer_id) REFERENCES CUSTOMER(customer_id)


In [28]:
RENT = [['inventory_id', 'customer_id', 'payment_id', 'rental_period', 'rental_start_date', 'rental_end_date', 
         'price_charged', 'overdue', 'overdue_duration', 'late_fee']]

tod = datetime.datetime.now()

for i in range(1,len(CUSTOMER)) :
    if fake.boolean(percent_borrowing) :
        rental_quantity = rng.integers(1,4)
        while rental_quantity > 0:
            # Determine rental period and start date 
            ## Length of the rental (1 to 7 days)
            rental_period = rng.integers(1, 8)

            ## Total rental time; may or may not exceed rental period
            total_rental_time = np.ceil(rng.normal(loc = 2, scale = .5))
            total_rental_time = total_rental_time if total_rental_time > 0 else 1
            rental_start_date = dt.datetime.today() - dt.timedelta(days = total_rental_time)
            rental_end_date = rental_start_date + dt.timedelta(days = float(rental_period))
            
            
            # Determine whether the rental is overdue
            overdue = False
            overdue_duration = 0

            ## If the projected end of the rental period is less than the current date, then it is overdue
            ## Projected end of rental period = rental start date + total rental time

            if rental_end_date < dt.datetime.today() :
                overdue = True
                overdue_duration = dt.datetime.today() - rental_end_date
  
            RENT.append([rng.integers(1, len(INVENTORY)), i, rng.integers(1,len(PAYMENT_METHOD)),
                         rental_period, rental_start_date, rental_end_date, 
                         rental_period * 1.00, int(overdue), overdue_duration, 0])
            
            rental_quantity = rental_quantity - 1

# Write to CSV
with open('Final Project/Phase 2/Fake Data/rent.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(RENT)

### Membership table
**Documentation**: `membership_status_desc` is a description of the membership tier level. `discount` is a float that can be multiplied by the base price to give the discounted price (essentially `discount` is $1-d$). `late fee` is a float the indicates how much a customer is charged per day the rental is returned late. 
- membership_status_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- membership_status_desc varchar(50) NOT NULL,
- discount float NOT NULL,
- late_fee float NOT NULL

In [24]:
MEMBERSHIP = [['membership_status_desc', 'discount', 'late_fee'],
              ['Guest', 1, 1.00],
              ['Premier Member', .95, .50],
              ['Silver Member', .90, .25],
              ['Gold Member', .75, .10],
              ['Platinum Member', .5, 0]]
# Write to CSV
with open('Final Project/Phase 2/Fake Data/membership.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(MEMBERSHIP)


### Orders table
**Documentation**:
- order_id int NOT NULL IDENTITY(1, 1) PRIMARY KEY,
- total_price float NOT NULL,
- vendor_id int NOT NULL,
- employee_id int NOT NULL,
- item_quantity int NOT NULL,
- item_price float NOT NULL,
- inventory_id int NOT NULL,
- FOREIGN KEY (vendor_id) REFERENCES VENDORS(vendor_id),
- FOREIGN KEY (employee_id) REFERENCES EMPLOYEES(employee_id),
- FOREIGN KEY (inventory_id) REFERENCES INVENTORY(inventory_id)

In [25]:
ORDERS = [['total_price', 'vendor_id', 'employee_id', 'item_quantity', 'item_price', 'inventory_id']]

for i in range(len(INVENTORY)) :
    n_aquisitions = int(rng.normal(loc=5, scale = 1))

    for j in range(n_aquisitions) :
        item_price = rng.uniform(2, 10)
        item_quantity = rng.integers(1,20)

        ORDERS.append([item_price * item_quantity, rng.integers(1,len(VENDORS)), rng.integers(1,len(EMPLOYEE)),
                       item_quantity, item_price, rng.integers(1,len(INVENTORY))])
        
# Write to CSV
with open('Final Project/Phase 2/Fake Data/orders.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ORDERS)